# Final Project - Braille Text Translator
## SCC0251 - Image Processing

### Students
* Caio Augusto Duarte Basso - NUSP 10801173
* Gabriel Garcia Lorencetti - NUSP 10691891
* Leonardo Rossi Luiz - NUSP 10851691
* Witor Matheus Alves de Oliveira - NUSP 10692190


### Main objective
The main objective of this project is to build a program able to recognize letters of a braille text present in an image digitally generated, i.e, given an input image, containing a text in braille, perform the translation to the alphabetic writing system.

### Iput images
The images used in this project were digitally generated, with the aid of a Braille letter dictionary and a code (which will be shown below) capable of generating Braille sentences, given an input.

### Importing required libraries

In [1]:
import numpy as np
import imageio
import scipy
import scipy.ndimage
import sys
import matplotlib.pyplot as plt
from skimage import morphology
from PIL import Image

### Generating a Braille Image
Given an input sentence, and with the help of the dictionary, the code generates an image containing the sentence in the Braille system.

In [2]:
# Gerando texto em braille para entrada do nosso problema
letters = [Image.open(x) for x in ['dictionary/a.png', 'dictionary/b.png', 'dictionary/c.png', 'dictionary/d.png',
                                   'dictionary/e.png', 'dictionary/f.png', 'dictionary/g.png', 'dictionary/h.png',
                                   'dictionary/i.png', 'dictionary/j.png', 'dictionary/k.png', 'dictionary/l.png',
                                   'dictionary/m.png', 'dictionary/n.png', 'dictionary/o.png', 'dictionary/p.png',
                                   'dictionary/q.png', 'dictionary/r.png', 'dictionary/s.png', 'dictionary/t.png',
                                   'dictionary/u.png', 'dictionary/v.png', 'dictionary/w.png', 'dictionary/x.png',
                                   'dictionary/y.png', 'dictionary/z.png', 'dictionary/_space.png'
                                  ]]

print("Type a word or sentence to create braille image.")
word = input().rstrip()

images = []
for letter in word:
    if letter != ' ':
        images.append(letters[ord(letter)-ord('a')])
    else: images.append(letters[26])
widths, heights = zip(*(i.size for i in images))

total_width = sum(widths)
max_height = max(heights)

new_im = Image.new('RGB', (total_width, max_height))

x_offset = 0
for im in images:
  new_im.paste(im, (x_offset,0))
  x_offset += im.size[0]

word_chunks = word.split(' ')
new_im.save(word_chunks[0]+'.png')

Type a word or sentence to create braille image.
hello


###  Preprocessing and quantisation
After performing the image generation, we will transform the image to grayscale using the Luminance method. Afterwards, as the image has only two colors, black and white, we will quantize the image to use only 1 bit, which will help us to make subsequent calculations more efficient.

In [3]:
def To_grayscale(img):    
    imgA = np.floor(img[:,:,0] * 0.299 + img[:,:,1] * 0.587 + img[:,:,2] * 0.114)
    imgA = imgA.astype('uint8')
    return imgA

In [4]:
def Quantisation (img, b):
    img = np.right_shift(img, 8-b)
    return img

### Euclidean distance calculation

In [5]:
def Difference(a, b):
    np.seterr(all='ignore')
    return np.sqrt(np.sum((a-b)**2))

In [6]:
def letter_representation(_img_ref, bits):
    ####### reading image #######
    dot_left = imageio.imread("dictionary/_left_dot.png")
    dot_right = imageio.imread("dictionary/_right_dot.png")
    ref_img = imageio.imread(_img_ref)
    #############################


    ###### Preprocessing and Quantisation ######
    grayscale_left = To_grayscale(dot_left)
    grayscale_right = To_grayscale(dot_right)
    grayscale_left = Quantisation(grayscale_left, bits)
    grayscale_right = Quantisation(grayscale_right, bits)

    g_ref_img = To_grayscale(ref_img)
    g_ref_img = Quantisation(g_ref_img, bits)

    grayscale_left = morphology.closing(grayscale_left, morphology.disk(4)).astype(np.uint8)
    grayscale_right = morphology.closing(grayscale_right, morphology.disk(4)).astype(np.uint8)
    g_ref_img = morphology.closing(g_ref_img, morphology.disk(4)).astype(np.uint8)
    ############################################


    ###### Finding the object #########
    N,M = g_ref_img.shape
    windows = np.empty((int(M/50)*6, 18, 25))

    a = 0
    for i in range (0, M-1, 50): # creating windows
            temp_wind = g_ref_img[:,i:i+50]
            n,m = temp_wind.shape
            windows[a] = temp_wind[:int(n/3),:int(m/2)] # 1
            windows[a+1] = temp_wind[:int(n/3),int(m/2):m] # 01
            windows[a+2] = temp_wind[int(n/3):int(2*n/3),:int(m/2)] # 00/1
            windows[a+3] = temp_wind[int(n/3):int(2*n/3),int(m/2):m] # 00/01
            windows[a+4] = temp_wind[int(2*n/3):n,:int(m/2)] # 00/00/1
            windows[a+5] = temp_wind[int(2*n/3):n,int(m/2):m] # 00/00/01
            a += 6

    
    min_dist = sys.maxsize
    close = -1
    text = [[0]*6]
    
    # calculating the distances and finding the closest window
    a = 0
    for y in range (int(M/50)):
        letter_value = np.zeros((1,6), dtype=np.int8)
        for i in range(6):
            if Difference(grayscale_left, windows[a+i]) <= 0.0001 or Difference(grayscale_right, windows[a+i]) <= 0.0001:
                letter_value[0][i] = 1

                
        text = np.append(text, letter_value, axis=0)
        a += 6
    
    return text

In [7]:
# Gerando matriz representante do texto de entrada
_img_ref = input().rstrip() # read reference image's name

bits = 1 # quantisation parameter

text_value = letter_representation(_img_ref, bits)[1:]
print("Valor do texto:")
print(text_value)

hello.png
Valor do texto:
[[1 0 1 1 0 0]
 [1 0 0 1 0 0]
 [1 0 1 0 1 0]
 [1 0 1 0 1 0]
 [1 0 0 1 1 0]]


In [8]:
# Gerando dicionario de cada letra com seu respectivo valor vetorial
dic = {}
alphabet = ['dictionary/a.png', 'dictionary/b.png', 'dictionary/c.png', 'dictionary/d.png', 'dictionary/e.png', 
            'dictionary/f.png', 'dictionary/g.png', 'dictionary/h.png', 'dictionary/i.png', 'dictionary/j.png', 
            'dictionary/k.png', 'dictionary/l.png', 'dictionary/m.png', 'dictionary/n.png', 'dictionary/o.png', 
            'dictionary/p.png', 'dictionary/q.png', 'dictionary/r.png', 'dictionary/s.png', 'dictionary/t.png', 
            'dictionary/u.png', 'dictionary/v.png', 'dictionary/w.png', 'dictionary/x.png', 'dictionary/y.png', 
            'dictionary/z.png'
           ]

for temp in alphabet:
    dic[temp[11]] = list(letter_representation(temp,1)[1])
dic[' '] = [0,0,0,0,0,0]

In [9]:
# Comparando valor do texto com o dicionario e formando resultado final
final_text = ""
for i in range(text_value.shape[0]):
    for key, values in dic.items():
        if(list(text_value[i]) == list(values)):
            final_text += key
            
print("Texto traduzido:", final_text)

Texto traduzido: hello
